# Layer-wise Relevance Propagation

### Different hyperparameters testing, over a Cifar10 CNN with MaxPooling Layers

#### Imports

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
#import time
#os.chdir('/home/ayudante/Desktop/Esteban/Code_vFloat32_TB')
import sequential32
import linear32 
import convolution32
import maxpool32 
import avgpool32
from heatmap import plot_relevances 
import input_data
import utils
from cifar10 import CIFAR10
import batchNormConv
import rotation

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2

%matplotlib inline

#### Some parameters

In [2]:
BATCH_SIZE = 50
#explanation of each one?
VALIDATION_PERIOD = 10000  # TODO set to 10k
ANNEALING_PERIOD = 100000  # TODO set to 100k
INITIAL_PATIENCE = 100000  # TODO set to 100k
PATIENCE_INCREMENT = 100000  # TODO set to 100k
SUMMARY_DIR = "/home/esteban/Esteban/TB/deep_hits_runs/batch%d_mi_val" % BATCH_SIZE

whole_data_path = '/home/esteban/deep_hits'

def get_train_tensors():
    data_path = whole_data_path+'/snr_train.tfrecord'
    feature = {'image_raw': tf.FixedLenFeature([], tf.string),
               'label': tf.FixedLenFeature([], tf.int64),
               'snr': tf.FixedLenFeature([], tf.string)}
    # Create a list of filenames and pass it to a queue
    filename_queue = tf.train.string_input_producer([data_path])
    # Define a reader and read the next record
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    # Decode the record read by the reader
    features = tf.parse_single_example(serialized_example, features=feature)
    # Convert the image data from string back to the numbers
    image = tf.decode_raw(features['image_raw'], tf.float32)

    # Cast label data into int32
    label = tf.cast(features['label'], tf.int32)
    
    # Cast snr data into int32
    snr = tf.decode_raw(features['snr'], tf.float64)
    
    # Reshape image data into the original shape
    image = tf.reshape(image, [21, 21, 4])
    
    snr = tf.reshape(snr, [1])

    # Any preprocessing here ...

    # Creates batches by randomly shuffling tensors
    images, labels, snrs= tf.train.batch([image, label, snr],
                                    batch_size=BATCH_SIZE,
                                    capacity=100000,
                                    num_threads=1)
    return images, labels, snrs



def get_validation_tensors():
    data_path = whole_data_path+'/snr_validation.tfrecord'
    feature = {'image_raw': tf.FixedLenFeature([], tf.string),
               'label': tf.FixedLenFeature([], tf.int64),
               'snr': tf.FixedLenFeature([], tf.string)}
    # Create a list of filenames and pass it to a queue
    filename_queue = tf.train.string_input_producer([data_path])
    # Define a reader and read the next record
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    # Decode the record read by the reader
    features = tf.parse_single_example(serialized_example, features=feature)
    # Convert the image data from string back to the numbers
    image = tf.decode_raw(features['image_raw'], tf.float32)

    # Cast label data into int32
    label = tf.cast(features['label'], tf.int32)
    
    # Cast snr data into int32
    snr = tf.decode_raw(features['snr'], tf.float64)
    
    # Reshape image data into the original shape
    image = tf.reshape(image, [21, 21, 4])
    
    snr = tf.reshape(snr, [1])

    # Any preprocessing here ...

    # Creates batches by randomly shuffling tensors
    images, labels, snrs= tf.train.batch([image, label, snr],
                                    batch_size=BATCH_SIZE,
                                    capacity=100000,
                                    num_threads=1)
    return images, labels, snrs


def get_test_tensors():
    data_path = whole_data_path+'/snr_test.tfrecord'
    feature = {'image_raw': tf.FixedLenFeature([], tf.string),
               'label': tf.FixedLenFeature([], tf.int64),
               'snr': tf.FixedLenFeature([], tf.string)}
    # Create a list of filenames and pass it to a queue
    filename_queue = tf.train.string_input_producer([data_path])
    # Define a reader and read the next record
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    # Decode the record read by the reader
    features = tf.parse_single_example(serialized_example, features=feature)
    # Convert the image data from string back to the numbers
    image = tf.decode_raw(features['image_raw'], tf.float32)

    # Cast label data into int32
    label = tf.cast(features['label'], tf.int32)
    
    # Cast snr data into int32
    snr = tf.decode_raw(features['snr'], tf.float64)
    
    # Reshape image data into the original shape
    image = tf.reshape(image, [21, 21, 4])
    
    snr = tf.reshape(snr, [1])

    # Any preprocessing here ...

    # Creates batches by randomly shuffling tensors
    images, labels, snrs= tf.train.batch([image, label, snr],
                                    batch_size=BATCH_SIZE,
                                    capacity=100000,
                                    num_threads=1)
    return images, labels, snrs


#### Data augmentation

In [3]:
def augment_with_rotations(img_batch):
    """ Receives a batch of 3-channel images and 
    returns a bigger batch (4 times bigger) with rotations
    in 90, 180 and 270 degrees. Be careful with the order
    of the samples in the output tensor
    """

    # Might be useful to modify parallel_iterations,
    # back_prop and swap_memory params of tf.map_fn
    images90 = tf.map_fn(lambda x: tf.image.rot90(x, k=1), img_batch)
    images180 = tf.map_fn(lambda x: tf.image.rot90(x, k=2), img_batch)
    images270 = tf.map_fn(lambda x: tf.image.rot90(x, k=3), img_batch)

    # TODO tf v1.0 has a different tf.concat call
    return tf.concat([img_batch,
                         images90,
                         images180,
                         images270], 0)

#### Instantiatite network model

This consider a pretreined model. To be loaded, weights must be in a 'weights2' folder, relative to this notebook.

In [4]:
def nn():
    return sequential32.Sequential([convolution32.Convolution(kernel_size=4,
                                                              output_depth=32, 
                                                              input_depth=4,
                                                              input_dim=27, act ='relu',
                                                              stride_size=1, pad='VALID'),
                                    
                       batchNormConv.BatchNorm(output_depth=32, phase_train= phase_train),
                                    
                       convolution32.Convolution(kernel_size=3, output_depth=32,
                                                 stride_size=1, act ='relu',
                                                 pad='SAME'),
                                    
                       batchNormConv.BatchNorm(output_depth=32, phase_train= phase_train),
                       
                       maxpool32.MaxPool(),

                       convolution32.Convolution(kernel_size=3, output_depth=64,
                                                 stride_size=1, act ='relu',
                                                 pad='SAME'),
                                    
                       batchNormConv.BatchNorm(output_depth=64, phase_train=phase_train ),
                                    
                       convolution32.Convolution(kernel_size=3, output_depth=64,
                                                 stride_size=1, act ='relu',
                                                 pad='SAME'),
                                    
                       batchNormConv.BatchNorm(output_depth=64, phase_train=phase_train ),
                                    
                       convolution32.Convolution(kernel_size=3, output_depth=64,
                                                 stride_size=1, act ='relu',
                                                 pad='SAME'),
                                    
                       batchNormConv.BatchNorm(output_depth=64, phase_train=phase_train ),
                                    
                       maxpool32.MaxPool(),
                                    
                       rotation.Rotation(rotation_num=4),
                       
                       linear32.Linear(64, act ='relu', keep_prob=keep_prob, 
                                       use_dropout = True),
                                               
                       linear32.Linear(64, act ='relu', keep_prob=keep_prob, 
                                       use_dropout = True),

                       linear32.Linear(2, act ='linear')])



#### Model I/O

In [5]:
#Dropout placeholder
keep_prob = tf.placeholder(tf.float32, name='keep-prob')
phase_train = tf.placeholder(tf.bool, shape=())

with tf.device('/cpu:0'):
    images, labels, snr = get_train_tensors()

    validation_images, validation_labels, validation_snr = get_validation_tensors()
    test_images, test_labels, test_snr = get_test_tensors()

    images = tf.placeholder_with_default(images,
                                         (None, 21, 21, 4),
                                         'images_placeholder')
    labels = tf.placeholder_with_default(labels,
                                         None,
                                         'labels_placeholder')
    one_hot_labels = tf.one_hot(labels, 2, dtype=tf.float32)
    augmented_input = augment_with_rotations(images)
    #augmented_input=images

    # from 21x21 stamps to 27x27 (NWHC)
    padded_input = tf.pad(augmented_input,
                          paddings=[
                              [0, 0],
                              [3, 3],
                              [3, 3],
                              [0, 0]])
    
    inp=padded_input
    y_=one_hot_labels

#Model instance
with tf.variable_scope('model'):
        #instanciate model
        net = nn()

        #feed-forward method and get score output
        score = net.forward(inp)
        score = tf.squeeze(score)

        #pass scores through softmax for network output ([0,1] probability)   
        y = tf.nn.softmax(score)

        #predicted classes
        y_pred_cls = tf.argmax(y, 1)

        #true classes
        y_true_cls = tf.argmax(y_, 1)
 

        
with tf.variable_scope('train'):
    
        #xentropy_loss = xentropy(fc_layer_12.output_tensor_without_activation,
         #                one_hot_labels)
    
        diff = tf.nn.softmax_cross_entropy_with_logits(logits=score, labels=y_)
        cost = tf.reduce_mean(diff)
        
        learning_rate = tf.Variable(0.04, trainable=False, collections=[])

        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        train_op = optimizer.minimize(cost)

Forward Pass ... 
------------------------------------------------- 
input:: [None, 27, 27, 4]
conv2d_1:: [None, 24, 24, 32]
batchNormConv_2:: [None, 24, 24, 32]
conv2d_3:: [None, 24, 24, 32]
batchNormConv_4:: [None, 24, 24, 32]
maxpool_5:: [None, 12, 12, 32]
conv2d_6:: [None, 12, 12, 64]
batchNormConv_7:: [None, 12, 12, 64]
conv2d_8:: [None, 12, 12, 64]
batchNormConv_9:: [None, 12, 12, 64]
conv2d_10:: [None, 12, 12, 64]
batchNormConv_11:: [None, 12, 12, 64]
maxpool_12:: [None, 6, 6, 64]
rotation_13:: [None, 9216]
linear_14:: [None, 64]
linear_15:: [None, 64]
linear_16:: [None, 2]
softmax:: [None, 2]

------------------------------------------------- 


#### Accuracy measurement

In [6]:
with tf.name_scope('accuracy'):  
        ## Performance measures
        
        #compare predictions
        with tf.name_scope('correct_prediction'):
            correct_prediction = tf.equal(y_pred_cls, y_true_cls)
        #get accuracy
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
accuracy_sum = tf.summary.scalar('accuracy', accuracy)

#### Other performance measures

In [7]:
with tf.name_scope('measures'):
    
    with tf.name_scope('values'):
            TP = tf.count_nonzero((y_pred_cls * y_true_cls))
            TN = tf.count_nonzero((y_pred_cls - 1) * (y_true_cls- 1))
            FP = tf.count_nonzero(y_pred_cls * (y_true_cls - 1))
            FN = tf.count_nonzero((y_pred_cls - 1) * y_true_cls)
            
    with tf.name_scope('accuracy_func'):
            acc_mes = (TP+TN)/(TP+TN+FN+FP)
        
    with tf.name_scope('precision_func'):
            prec_mes = TP/(TP+FP)
            
    with tf.name_scope('recall_func'):
            rec_mes = TP/(TP+FN)      
            
    with tf.name_scope('f1_func'):
            f1_mes = 2 * prec_mes * rec_mes / (prec_mes + rec_mes) 

#### LRP with different parameters

In [8]:
config = tf.ConfigProto(allow_soft_placement = True)
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)


#train_writer = tf.summary.FileWriter(os.path.join(SUMMARY_DIR, 'train'),
#                                     sess.graph,
#                                     max_queue=1000)

#validation_writer = tf.summary.FileWriter(os.path.join(SUMMARY_DIR, 'validation'))

metrics = (cost, accuracy)
metrics_names = ('xentropy', 'accuracy')
metric_summary_list = []

for metric, name in zip(metrics, metrics_names):
    summary = tf.summary.scalar(name, metric)
    metric_summary_list.append(summary)

merged_summaries = tf.summary.merge(metric_summary_list)

#saver = tf.train.Saver()

init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess.run(init_op)
# Create a coordinator and run all QueueRunner objects
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

# for batch_index in range(5):
#     img, lbl = sess.run([images, labels])
#     # img = img.astype(np.uint8)
#     for j in range(4):
#         plt.subplot(2, 2, j + 1)
#         plt.imshow(img[0, :, :, j])
#         plt.title('bogus' if lbl[0] == 0 else 'real')
#     plt.show()




In [9]:
def validate(current_iteration, current_patience, best_model, stopping_criteria_model):
    if current_iteration % VALIDATION_PERIOD != 0:
        return current_patience

    metric_data = {}
    for metric in metrics:
        metric_data[metric] = {
            'values_per_batch': [],
            'batch_mean': None
        }

    # TODO check that validation batches are right. UPDATE: Looks good.
    for val_batch in range(100000 // BATCH_SIZE):
        images_array, labels_array = sess.run((
            validation_images,
            validation_labels))

        metrics_value = sess.run(metrics,
                                 feed_dict={
                                     keep_prob: 1.0,
                                     phase_train: False,
                                     images: images_array,
                                     labels: labels_array
                                 })
        for metric, value in zip(metrics, metrics_value):
            metric_data[metric]['values_per_batch'].append(value)

    for metric, name in zip(metrics, metrics_names):
        metric_data[metric]['batch_mean'] = np.array(metric_data[metric]['values_per_batch']).mean()
        summary = tf.Summary()
        summary.value.add(tag=name, simple_value=float(metric_data[metric]['batch_mean']))
        #validation_writer.add_summary(summary, current_iteration)

    accuracy_mean = metric_data[accuracy]['batch_mean']
    # Check best model
    if accuracy_mean > best_model['accuracy']:
        best_model['accuracy'] = accuracy_mean
        best_model['iteration'] = current_iteration
        print("New best model: Accuracy %.4f @ it %d" % (
            best_model['accuracy'],
            best_model['iteration']
        ))
        ckpt_dir = os.path.join(SUMMARY_DIR, 'ckpt_files')
        if not os.path.exists(ckpt_dir):
            os.makedirs(ckpt_dir)
        #saver.save(sess,
        #           os.path.join(ckpt_dir, '%d_iters.ckpt' % current_iteration))

    # update patience
    if (1.0-accuracy_mean) < 0.99*(1.0-stopping_criteria_model['accuracy']):
        stopping_criteria_model['accuracy'] = accuracy_mean
        stopping_criteria_model['iteration'] = current_iteration
        new_patience = current_iteration + PATIENCE_INCREMENT
        if new_patience > current_patience:
            print("Patience increased to %d because of model with accuracy %.4f @ it %d" % (
                new_patience,
                stopping_criteria_model['accuracy'],
                stopping_criteria_model['iteration']
            ))
            return new_patience
        else:
            # Case when new_patience is smaller than INITIAL_PATIENCE
            return current_patience
    else:
        return current_patience


def test():
    # It does not load best params, just uses the current ones.
    losses = []
    accuracies = []
    precisions = []
    recalls = []
    f1s = []
    for test_batch in range(100000 // BATCH_SIZE):
        images_array, labels_array = sess.run((
            test_images,
            test_labels))
        loss_val, acc_val, prec, rec, f1 = sess.run((cost, acc_mes, prec_mes, rec_mes, f1_mes),
                                     feed_dict={
                                         keep_prob: 1.0,
                                         images: images_array,
                                         labels: labels_array,
                                         phase_train: False
                                     })
        losses.append(loss_val)
        accuracies.append(acc_val)
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        
        #print(test_batch)
    loss_mean = np.array(losses).mean()
    accuracy_mean = np.array(accuracies).mean()
    precision_mean = np.array(precisions).mean()
    recall_mean = np.array(recalls).mean()
    f1_mean = np.array(f1s).mean()
    

    return loss_mean, accuracy_mean, precision_mean, recall_mean, f1_mean


def update_learning_rate(global_step):
    sess.run(tf.assign(learning_rate,
                       0.04/(2.0**(global_step//ANNEALING_PERIOD))))
    lr_value = sess.run(learning_rate)
    print("Iteration %d. Learning rate: %.4f" % (global_step, lr_value))

def train_from_scratch():
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(init_op)
    global_step = 0
    patience = INITIAL_PATIENCE
    best_model = {
        'iteration': 0,
        'accuracy': 0.5
    }
    stopping_criteria_model = {
        'iteration': 0,
        'accuracy': 0.5
    }
    while global_step < patience:
        if global_step % ANNEALING_PERIOD == 0:
            update_learning_rate(global_step)
        summaries_output, _ = sess.run((merged_summaries,
                                        train_op),
                                       feed_dict={
                                           phase_train: True,
                                           keep_prob: 0.5
                                       })
        global_step += 1
        #train_writer.add_summary(summaries_output, global_step)
        patience = validate(global_step, patience, best_model, stopping_criteria_model)

    #saver.restore(sess, os.path.join(
    #    SUMMARY_DIR,
    #    'ckpt_files/%d_iters.ckpt' % best_model['iteration']))
    test_loss, test_accuracy, test_prec, test_rec, test_f1 = test()
    print("Best model @ it %d.\nValidation accuracy %.5f, Test accuracy %.5f" % (
        best_model['iteration'],
        best_model['accuracy'],
        test_accuracy
    ))
    print("Test loss %.5f" % test_loss)
    
    return test_accuracy, test_prec, test_rec, test_f1

# Stop the threads
#coord.request_stop()

# Wait for threads to stop
#coord.join(threads)
#sess.close()

def eval_set(set_images, set_labels):
    # It does not load best params, just uses the current ones.
    losses = []
    accuracies = []
    precisions = []
    recalls = []
    f1s = []
    for test_batch in range(100000 // BATCH_SIZE):
        images_array, labels_array = sess.run((
            set_images,
            set_labels))
        loss_val, acc_val, prec, rec, f1 = sess.run((cost, acc_mes, prec_mes, rec_mes, f1_mes),
                                     feed_dict={
                                         keep_prob: 1.0,
                                         images: images_array,
                                         labels: labels_array,
                                         phase_train: False
                                     })
        losses.append(loss_val)
        accuracies.append(acc_val)
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        
        #print(test_batch)
    loss_mean = np.array(losses).mean()
    accuracy_mean = np.array(accuracies).mean()
    precision_mean = np.array(precisions).mean()
    recall_mean = np.array(recalls).mean()
    f1_mean = np.array(f1s).mean()
    

    return loss_mean, accuracy_mean, precision_mean, recall_mean, f1_mean


In [10]:
acc_ls = []
prec_ls = []
rec_ls = []
f1_ls = []
#models_test_probs = []

acc_ls_val = []
prec_ls_val = []
rec_ls_val = []
f1_ls_val = []


#models_val_probs = []
#models_val_lbls = []
#models_val_snr = []

for i in range(6):
    
    print("\nModel:", i)
    test_accuracy, test_prec, test_rec, test_f1 = train_from_scratch()
    
    #probs = getParams4Metrics(test_images, test_labels)   
    #models_test_probs.append(probs)
    
    acc_ls.append(test_accuracy)
    prec_ls.append(test_prec)
    rec_ls.append(test_rec)
    f1_ls.append(test_f1)
    
    _, val_accuracy, val_prec, val_rec, val_f1 = eval_set(validation_images, validation_labels)
    
    #lbls, probs, snrs = getParams4Metrics(validation_images, validation_labels, validation_snr)   
    #models_val_probs.append(probs)
    #models_val_lbls.append(lbls)
    #models_val_snr.append(snrs)
    
    acc_ls_val.append(val_accuracy)
    prec_ls_val.append(val_prec)
    rec_ls_val.append(val_rec)
    f1_ls_val.append(val_f1)


Model: 0
Iteration 0. Learning rate: 0.0400
New best model: Accuracy 0.9904 @ it 10000
Patience increased to 110000 because of model with accuracy 0.9904 @ it 10000
New best model: Accuracy 0.9919 @ it 30000
Patience increased to 130000 because of model with accuracy 0.9919 @ it 30000
New best model: Accuracy 0.9933 @ it 60000
Patience increased to 160000 because of model with accuracy 0.9933 @ it 60000
New best model: Accuracy 0.9934 @ it 90000
Patience increased to 190000 because of model with accuracy 0.9934 @ it 90000
Iteration 100000. Learning rate: 0.0200
New best model: Accuracy 0.9939 @ it 140000
Patience increased to 240000 because of model with accuracy 0.9939 @ it 140000
New best model: Accuracy 0.9943 @ it 180000
Patience increased to 280000 because of model with accuracy 0.9943 @ it 180000
Iteration 200000. Learning rate: 0.0100
New best model: Accuracy 0.9946 @ it 220000
Patience increased to 320000 because of model with accuracy 0.9946 @ it 220000
New best model: Accura

New best model: Accuracy 0.9923 @ it 40000
Patience increased to 140000 because of model with accuracy 0.9923 @ it 40000
New best model: Accuracy 0.9930 @ it 70000
Patience increased to 170000 because of model with accuracy 0.9930 @ it 70000
New best model: Accuracy 0.9933 @ it 80000
Patience increased to 180000 because of model with accuracy 0.9933 @ it 80000
Iteration 100000. Learning rate: 0.0200
New best model: Accuracy 0.9935 @ it 110000
Patience increased to 210000 because of model with accuracy 0.9935 @ it 110000
New best model: Accuracy 0.9942 @ it 120000
Patience increased to 220000 because of model with accuracy 0.9942 @ it 120000
New best model: Accuracy 0.9944 @ it 140000
Patience increased to 240000 because of model with accuracy 0.9944 @ it 140000
New best model: Accuracy 0.9946 @ it 170000
Patience increased to 270000 because of model with accuracy 0.9946 @ it 170000
Iteration 200000. Learning rate: 0.0100
New best model: Accuracy 0.9948 @ it 230000
Patience increased to

In [11]:
#import numpy as np
#acc=[0.99477994, 0.99450999, 0.99502999, 0.99476004, 0.99533999, 0.99461997]
def getMetricVal(metric_ls, metric_name):
    metric_mean = np.array(metric_ls).mean()*100
    metric_std = np.array(metric_ls).std()*100
    print("%s %.2f +/- %.2f" % (
        metric_name,
        np.array(metric_ls).mean()*100,
        np.array(metric_ls).std()*100
    ))
    return metric_mean, metric_std

print("Test Metrics\n")
_,_=getMetricVal(acc_ls, 'Accuracy')

_,_=getMetricVal(prec_ls, 'Precision')

_,_=getMetricVal(rec_ls, 'Recall')

_,_=getMetricVal(f1_ls, 'F1 Score')
    

Test Metrics

Accuracy 99.46 +/- 0.09
Precision 99.33 +/- 0.29
Recall 99.62 +/- 0.16
F1 Score 99.46 +/- 0.08


In [12]:
print("Val Metrics\n")
_,_=getMetricVal(acc_ls_val, 'Accuracy')

_,_=getMetricVal(prec_ls_val, 'Precision')

_,_=getMetricVal(rec_ls_val, 'Recall')

_,_=getMetricVal(f1_ls_val, 'F1 Score')

Val Metrics

Accuracy 99.43 +/- 0.08
Precision 99.34 +/- 0.27
Recall 99.54 +/- 0.19
F1 Score 99.43 +/- 0.07


In [13]:
print(acc_ls_val, 'Accuracy')

print(prec_ls_val, 'Precision')

print(rec_ls_val, 'Recall')

print(f1_ls_val, 'F1 Score')

[0.99479999999999991, 0.99264000000000008, 0.99397999999999997, 0.99487000000000014, 0.99473, 0.99459000000000009] Accuracy
[0.99395720717659874, 0.98793482036109348, 0.99578076034651863, 0.99414864869885333, 0.99596652471995939, 0.99278772671147986] Precision
[0.99589778551344188, 0.99807227996727832, 0.99236436302202191, 0.99583463489713808, 0.99364815430607656, 0.99674055584284627] Recall
[0.99481014197114537, 0.99278797259948004, 0.99386957963249845, 0.994864036375953, 0.99467523968813809, 0.99463039448250046] F1 Score


In [14]:
print(acc_ls, 'Accuracy')

print(prec_ls, 'Precision')

print(rec_ls, 'Recall')

print(f1_ls, 'F1 Score')

[0.995, 0.99270000000000003, 0.99467000000000005, 0.99517000000000011, 0.99523000000000006, 0.99482999999999999] Accuracy
[0.99345943704021278, 0.98739406791458872, 0.99556617699775662, 0.99410745218978036, 0.99621667910853862, 0.99293294972996682] Precision
[0.99685049971575113, 0.99872167830656211, 0.99392968555763361, 0.99644723426679305, 0.99440928135411222, 0.99695568118594569] Recall
[0.9950237611678826, 0.99282049161218766, 0.99462921874496169, 0.99516819050430949, 0.99520257555570768, 0.99482107013845578] F1 Score


In [16]:
W,B = net.getWeights()
weights, biases = sess.run([W,B], feed_dict={phase_train: False})

layer without params
layer without params
layer without params


In [17]:
def saveWeights(Ncnn, Nfc, weights, biases):
    for i in range(Ncnn):
        np.save('weights/deephitsRotBN/CNN'+str(i+1)+'-W.npy', np.array(weights[i]))
        np.save('weights/deephitsRotBN/CNN'+str(i+1)+'-B.npy', np.array(biases[i]))

    for i in range(Nfc):
        np.save('weights/deephitsRotBN/FC'+str(i+1)+'-W.npy', np.array(weights[i+Ncnn]))
        np.save('weights/deephitsRotBN/FC'+str(i+1)+'-B.npy', np.array(biases[i+Ncnn]))

saveWeights(10,3,weights,biases)

In [18]:
beta, gamma, mean, var = net.getBNparams()
b,g,m,v = sess.run([beta, gamma, mean, var], feed_dict={phase_train: False})

not a BN layer
not a BN layer
not a BN layer
not a BN layer
not a BN layer
not a BN layer
not a BN layer
not a BN layer
not a BN layer
not a BN layer
not a BN layer


In [19]:
def saveBNparams(Nbn, b,g,m,v):
    for i in range(Nbn):
        np.save('weights/deephitsRotBN/param'+str(i+1)+'-beta.npy', np.array(b[i]))
        np.save('weights/deephitsRotBN/param'+str(i+1)+'-gamma.npy', np.array(g[i]))
        np.save('weights/deephitsRotBN/param'+str(i+1)+'-mean.npy', np.array(m[i]))
        np.save('weights/deephitsRotBN/param'+str(i+1)+'-var.npy', np.array(v[i]))


saveBNparams(5, b,g,m,v)